In [1]:
print('hello')

hello


In [4]:
import torch
import intel_extension_for_pytorch as ipex
print(torch.__version__)

2.8.0+xpu


In [2]:
with open('frankenstein.txt', 'r', encoding='utf-8') as f:
    text=f.read()
    chars = sorted(list(set(text)))
print(text[:200])
print(f'Total characters: {chars}')

Letter 1

_To Mrs. Saville, England._


St. Petersburgh, Dec. 11th, 17—.


You will rejoice to hear that no disaster has accompanied the
commencement of an enterprise which you have regarded with suc
Total characters: ['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”', '\ufeff']


In [6]:
stringtoint={ch:i for i,ch in enumerate(chars)}
inttostring={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stringtoint[c] for c in s]
decode=lambda l: ''.join([inttostring[i] for i in l])

print(encode('hello world'))
print(decode(encode('hello world')))

[54, 51, 58, 58, 61, 1, 69, 61, 64, 58, 50]
hello world


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([418920]) torch.int64
tensor([83, 32, 51, 66, 66, 51, 64,  1,  9,  0,  0, 46, 39, 61,  1, 33, 64, 65,
         7,  1, 38, 47, 68, 55, 58, 58, 51,  5,  1, 25, 60, 53, 58, 47, 60, 50,
         7, 46,  0,  0,  0, 38, 66,  7,  1, 36, 51, 66, 51, 64, 65, 48, 67, 64,
        53, 54,  5,  1, 24, 51, 49,  7,  1,  9,  9, 66, 54,  5,  1,  9, 15, 78,
         7,  0,  0,  0, 43, 61, 67,  1, 69, 55, 58, 58,  1, 64, 51, 56, 61, 55,
        49, 51,  1, 66, 61,  1, 54, 51, 47, 64])
